In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
import xgboost as xgb
import catboost as catb
import lightgbm as lgbmfrom
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
import xgboost as xgb
import catboost as catb
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from sklearn.base import clone
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from vecstack import stacking
from sklearn.ensemble import StackingClassifier

In [24]:
# генераторы признаков квадраты и логорифмы
def create_poly_features(data, num_features):
    poly_features = np.power(data[num_features], 2)
    poly_features = poly_features.rename(columns=dict(zip(data.columns, [col+' sqr' for col in data.columns])))
    return data.join(poly_features)

def create_log_features(data, num_features):
    log_features = np.log(data[num_features], where=data[num_features]>0)
    log_features = log_features.rename(columns=dict(zip(data.columns, [col+'_log' for col in data.columns])))
    return data.join(log_features)

In [25]:
def correct_value(data, target, value):
    data.loc[data[target] < value, target] = data[target].median()
    return data

In [26]:
# Подготовим функцию обработки дата сета
def job(data):
      # Заменим тех у кого не измерили глюкозу на средние значения
      data = correct_value(data, 'Glucose', 60)
      # Установим средние значения для кровянного давления, у кого не измерили
      data = correct_value(data, 'BloodPressure', 60)
      # Установим средние значения тех у кого индекс массы тела меньше 18
      data = correct_value(data,'BMI', 18)
      # откорректируем толщину кожы в 5 мм так как этот фактор влияет на усвоение инсулина, и его нужно было измерять, тут явно пропуски
      data = correct_value(data,'SkinThickness', 5)
      # Откорректируем нижнюю границу нормы по инсулину
      data = correct_value(data,'Insulin', 15)

      #data=data.drop(['BloodPressure', 'SkinThickness', 'Insulin'], axis=1)
      
      # Добавим логарифмы и квадраты для числовых переменных
      data = create_poly_features(data, num_features)
      data = create_log_features(data, num_features)
      return data

In [27]:
data = pd.read_csv('./data/train.csv')
num_features = ['Pregnancies', 'Glucose', 'BMI', 'Age'] # Insulin
data = job(data)
data.to_pickle('work_data.pk2')